In [1]:
import tensorflow as tf
physicalDevices = tf.config.list_physical_devices('GPU') 
if physicalDevices:
    tf.config.experimental.set_memory_growth(physicalDevices[0], True)
from tensorflow import keras
from tensorflow.keras import layers
from data_preprocessor import DataGenerator, GESTURES

import datetime

In [6]:
batch_size=64
epochs=30
input_shape = (20,21,3)
classes = len(GESTURES)
trainGen = DataGenerator('gesture_dataset.h5')

In [7]:
inputs = layers.Input(shape=input_shape)
x = inputs
x = layers.TimeDistributed(layers.Flatten())(x)
x = layers.TimeDistributed(layers.Dropout(0.5))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.2))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.2))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.2))(x)
x = layers.GRU(32)(x)
output = layers.Dense(classes, activation='softmax',name='Output')(x)
model = keras.models.Model(inputs, output)
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 20, 21, 3)]       0         
_________________________________________________________________
time_distributed_8 (TimeDist (None, 20, 63)            0         
_________________________________________________________________
time_distributed_9 (TimeDist (None, 20, 63)            0         
_________________________________________________________________
time_distributed_10 (TimeDis (None, 20, 64)            4096      
_________________________________________________________________
time_distributed_11 (TimeDis (None, 20, 64)            0         
_________________________________________________________________
time_distributed_12 (TimeDis (None, 20, 64)            4160      
_________________________________________________________________
time_distributed_13 (TimeDis (None, 20, 64)            0   

In [8]:
model.fit_generator(generator=trainGen,epochs=epochs)

In [9]:
model.save("saved_models/MODEL-{}".format(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")), save_format = "tf")